In [192]:
from Dataload import dataload

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils

from src import detect_faces, show_bboxes
from PIL import Image
import numpy as np
#np_load_old = np.load

#np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
from torchvision import transforms

import pandas as pd
from module import densenet

from tqdm.notebook import tqdm

In [200]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

class Video_Frame_Data(Dataset):
    def __init__(self, device,csv_file,base_path_v=None,base_path_a=None,strict_num=16,strict_name=True,name_format=9,transform=None,embedding=False):
        self._table = pd.read_csv(csv_file,delimiter=' ')
        self.strict_num = strict_num
        self._base_path_v=base_path_v
        self._base_path_a=base_path_a
        self.device=device
        self.embedding=embedding
        if strict_name:
            self.name_format=name_format
            


    def __len__(self):
        return len(self._table)

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path_v,self._table.Vid_name[idx])
        first=True
        audio_img=np.zeros((1,1))
        
        frame_len=len(os.listdir(folder_name))
        frame_list=[]
        if frame_len<self.strict_num:
            for index_0 in range(frame_len):
                frame_path=os.path.join(folder_name,os.listdir(folder_name)[index_0])
                tempimg=Image.open(frame_path)       
                tempimg=frame(tempimg,frame_path,device=self.device,embedding=self.embedding)
                frame_list.append(tempimg)
        else:    
            frame_index=(np.linspace(0,frame_len-1,self.strict_num,dtype=int))

            for index_2 in tqdm(frame_index):
                frame_path=os.path.join(folder_name,os.listdir(folder_name)[index_2])
                tempimg=Image.open(frame_path)
                tempimg=frame(tempimg,frame_path,device=self.device,embedding=self.embedding)
                frame_list.append(tempimg)

        labels = torch.from_numpy(np.array(self._table.Label[idx]))-1
            

        return (frame_list,audio_img,labels)
    

In [201]:
class frame_list():
    def __init__(frame_list):
        self.frame_list=frame_list

class frame():
    def __init__(self,frame_img,frame_name,max_face=5,embedding=False,frame_csv=None,face_csv=None,device=None):
        self.frame_img=frame_img
        self._file_name=frame_name
        self.transform_face=transforms.Compose([
                                 transforms.Resize((32,32)), 
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
        
        self.transform_frame=transforms.Compose([
                     transforms.Resize((256,256)),
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])

        
        def face_extraction(img,max_face):
            bounding_boxes, landmarks = detect_faces(img)
            img_list=[]
            for box_index,(left,right,up,bottom,_) in enumerate(bounding_boxes):
                cropped_img=img.crop((left,right,up,bottom))
                img_list.append(self.transform_face(cropped_img))
                if len(img_list)==max_face:
                    break
            while len(img_list) !=max_face:
                END_PAD= Image.new(mode = "RGB", size = (256, 256), color =(0, 0, 0))
                img_list.append(self.transform_face(END_PAD))
            img_list=torch.stack(img_list)

            return img_list
        self.faces=face_extraction(self.frame_img,max_face)


        
        self.frame_img=self.transform_frame(frame_img)
        if embedding:
            model=densenet.densenet121(pretrained=True)
            if device:
                model=model.to(device)
                self.frame_img=self.frame_img.unsqueeze(0).to(device)    
            self.pre_embed=model(self.frame_img).detach().cpu()
        
    
        
        
    
        
        

In [202]:
import os
from constant import EMOTIPATH


Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video=os.path.join(EMOTIPATH,"Val")


train_data=Video_Frame_Data(device,Train_label,base_path_v=Train_video,strict_num=5,embedding=True)
#valid_data=Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=5)

In [209]:
test=train_data[0][0][0].pre_embed


In [212]:
test

tensor([[-9.1954e-01, -2.9284e-01, -5.4782e-01, -1.5645e+00, -8.8902e-01,
         -2.1535e-01, -8.4027e-01,  6.3220e-01,  1.7955e-01, -6.3220e-01,
         -7.1697e-01, -8.1826e-01, -6.8394e-01, -9.3698e-01, -8.8048e-01,
         -6.1644e-01, -7.8646e-01, -2.8091e-01, -5.2236e-01, -7.8809e-01,
         -1.5594e+00, -8.0342e-01, -1.4142e+00, -3.1372e-02, -1.0248e+00,
         -5.9008e-01, -5.3709e-01, -5.3182e-01, -6.5277e-01, -4.6452e-01,
         -1.0874e+00, -7.2006e-01, -4.1613e-01, -7.3615e-01, -1.5370e-01,
         -5.3823e-01,  8.6620e-01, -3.4296e-01, -2.9759e-01,  3.1156e-01,
         -7.8097e-01, -5.8767e-01, -6.9279e-01, -4.3846e-01, -5.7265e-01,
         -4.4764e-01, -7.5767e-01, -2.7972e-01, -1.1850e+00, -9.6885e-01,
         -4.7003e-01,  4.0420e-01, -3.5443e-01, -4.2280e-01, -2.3904e-01,
         -9.6374e-01, -3.4387e-01, -1.0692e+00, -4.8043e-01,  2.9182e-02,
          1.0401e+00, -9.4923e-02,  2.4767e-02,  1.5344e-01, -4.6428e-01,
          1.1833e-01, -1.7312e-01, -3.

In [204]:
train_dataloader = DataLoader(train_data, batch_size=32
                       , num_workers=0,shuffle=True)

In [103]:
class Video_Frame_Data(Dataset):
    def __init__(self, csv_file,base_path_v=None,base_path_a=None,strict_num=16,strict_name=True,name_format=9,transform=None):
        self._table = pd.read_csv(csv_file,delimiter=' ')
        self.strict_num = strict_num
        self._base_path_v=base_path_v
        self._base_path_a=base_path_a
        if strict_name:
            self.name_format=name_format

        self.transform=transforms.Compose([
                     transforms.Resize((256,256)),
                     #transforms.Grayscale(),
                     transforms.ToTensor(),   
                     #transforms.Normalize((0.5, ), (0.5, ))])
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
    def __len__(self):
        return len(self._table)

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path_v,self._table.Vid_name[idx])
        first=True
        audio_img=np.zeros((1,1))
        
        
        ###Read Frames
        START_PAD = Image.new(mode = "RGB", size = (256, 256), color = (255, 255, 255) )
        END_PAD= Image.new(mode = "RGB", size = (256, 256), color =(0, 0, 0))
        
        img=self.transform(START_PAD).unsqueeze(0)
        END_img=self.transform(END_PAD).unsqueeze(0)
        frame_len=len(os.listdir(folder_name))
        
        if frame_len<self.strict_num:
            for index_0 in range(frame_len):
                tempimg=Image.open(os.path.join(folder_name,os.listdir(folder_name)[index_0]))#.convert('L')       
                tempimg=self.transform(tempimg)
                tempimg=tempimg.unsqueeze(0)
                if first:
                    img=tempimg
                    first=False
                else:
                    img=torch.cat((img,tempimg))
            for index_1 in range(self.strict_num-2-frame_len):
                img=torch.cat((img,END_img))
                
        else:    
            frame_index=(np.linspace(0,frame_len-1,self.strict_num,dtype=int))

            for index_2 in frame_index:
                tempimg=Image.open(os.path.join(folder_name,os.listdir(folder_name)[index_2]))#.convert('L')
                
                tempimg=self.transform(tempimg)
                tempimg=tempimg.unsqueeze(0)
                if first:
                    img=tempimg
                    first=False
                else:
                    img=torch.cat((img,tempimg))

        labels = torch.from_numpy(np.array(self._table.Label[idx]))-1
            
     
            
        #img=torch.cat((img,END_img))
       # img=img[1: -1]
        return (img,audio_img,labels)
    